In [ ]:
#https://gist.github.com/csaftoiu/9fccaf47fd8f96cd378afd8fdd0d63c1

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


Using TensorFlow backend.
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\d

In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models_2')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
                                                                            #I am using the old version here

In [ ]:

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 18s 12ms/step - loss: 1.8810 - acc: 0.3084 - val_loss: 1.5643 - val_acc: 0.4340
Epoch 2/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.5963 - acc: 0.4175 - val_loss: 1.4028 - val_acc: 0.4893
Epoch 3/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.4779 - acc: 0.4640 - val_loss: 1.3414 - val_acc: 0.5209
Epoch 4/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.3913 - acc: 0.4979 - val_loss: 1.3049 - val_acc: 0.5318
Epoch 5/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.3280 - acc: 0.5261 - val_loss: 1.2253 - val_acc: 0.5581
Epoch 6/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2705 - acc: 0.5479 - val_loss: 1.1290 - val_acc: 0.5982
Epoch 7/100
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2145 - acc: 0.5697 - val_loss: 1.0791 - val_acc: 0.6151
Epoch 

In [21]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_json = model.to_json()
model_json_path = os.path.join(save_dir, model_name.split('.')[0] + '.json')
with open(model_json_path, 'w') as json_file:
    json_file.write(model_json)
    
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\Frederico\Desktop\rodando\saved_models\keras_cifar10_trained_model.h5 


In [ ]:
# CARREGANDO A PARTIR DO MODELO DEFINIDO LA EM CIMA

In [16]:

model_construcao.load_weights("saved_models/keras_cifar10_trained_model.h5")
model_construcao.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])    

In [17]:
# Score trained model.
scores = model_construcao.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 80us/step
Test loss: 11.070283121490478
Test accuracy: 0.3122


In [3]:
from keras.models import model_from_json
# load json and create model
json_file = open('saved_models/keras_cifar10_trained_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models/keras_cifar10_trained_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
                                                                            #I am using the old version here
# Let's train the model using RMSprop
loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])    

In [5]:
# Score trained model. # era 0.78
scores = loaded_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 170us/step
Test loss: 11.070283121490478
Test accuracy: 0.3122
